In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/saved-model/checkpoint0.pth
/kaggle/input/saved-model/checkpoint0 (1).pth
/kaggle/input/saved-model/noisy_model_final/fingerprint.pb
/kaggle/input/saved-model/noisy_model_final/saved_model.pb
/kaggle/input/saved-model/noisy_model_final/keras_metadata.pb
/kaggle/input/saved-model/noisy_model_final/variables/variables.index
/kaggle/input/saved-model/noisy_model_final/variables/variables.data-00000-of-00001
/kaggle/input/saved-model/24_05_2023_08_40_46/fingerprint.pb
/kaggle/input/saved-model/24_05_2023_08_40_46/saved_model.pb
/kaggle/input/saved-model/24_05_2023_08_40_46/keras_metadata.pb
/kaggle/input/saved-model/24_05_2023_08_40_46/variables/variables.index
/kaggle/input/saved-model/24_05_2023_08_40_46/variables/variables.data-00000-of-00001
/kaggle/input/project-test-dataset/X/Dhash0.npy
/kaggle/input/project-test-dataset/X/E0.npy
/kaggle/input/project-test-dataset/X/E1.npy
/kaggle/input/project-test-dataset/X/B0.npy
/kaggle/input/project-test-dataset/X/Chash1.npy
/kaggl

In [2]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import os
import IPython.display as ipd
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score
from torch.utils.data import DataLoader, Dataset

import torch
from torcheval.metrics import MulticlassF1Score
from torchsummary import summary


In [5]:
a = []
os.chdir("/kaggle/input/project-train-spectrograms")
# names = ["B0.npy", "C0.npy", "C#0.npy", "D0.npy", "D#0.npy", "E0.npy"]
names = os.listdir()
for name in names:
    print(name)
    a.append(torch.Tensor(np.load("/kaggle/input/project-train-spectrograms/"+name)))
    
a = torch.cat(a, axis=0)

Dhash0.npy
E0.npy
E1.npy
B0.npy
Chash1.npy
Chash0.npy
C0.npy
D0.npy
D1.npy
Dhash1.npy
C1.npy
B1.npy


In [6]:
b = []

for name in names:
    print(name)
    l = []
    t1 = np.load("/kaggle/input/project-train-labels/"+name)
    for ele in t1:
        l.append(torch.unsqueeze(torch.Tensor(ele.T), 0))
    l = torch.cat(l, axis=0)
    b.append(l)
    del(l)
    del(t1)

# for name in names:
#   print(name)
#   b.append(np.load("/content/drive/MyDrive/UGP/Y/"+name))
b = torch.cat(b, axis = 0)

Dhash0.npy
E0.npy
E1.npy
B0.npy
Chash1.npy
Chash0.npy
C0.npy
D0.npy
D1.npy
Dhash1.npy
C1.npy
B1.npy


In [7]:
b.shape

torch.Size([1200, 1723, 6])

In [8]:
if torch.cuda.is_available():
    a = a.cuda()
    b = b.cuda()

In [9]:
b = b[:, :, [5, 0, 1, 2, 3, 4]]

In [10]:
# import seaborn as sns
# sns.heatmap(b[0])

In [11]:

epochs = 50
num_classes = 6
batch_size = 64
rnn_sizes = [16]
dropout_rate = 0.5



class MyModel(nn.Module):
    def __init__(self, num_classes, rnn_sizes, dropout_rate):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding='same')
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding='same')
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3, 3), padding='same')
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=(3, 3), padding='same')
        self.batch_norm4 = nn.BatchNorm2d(256)

        rnn_layers = []
        input_size = 2048  # Update the input size based on the shape after pooling
        for rnn_size in rnn_sizes:
            rnn_layers.append(nn.GRU(input_size, rnn_size, bidirectional=True, batch_first = True))
            input_size = rnn_size * 2

        self.rnn_layers = nn.ModuleList(rnn_layers)
        self.dropout = nn.Dropout(dropout_rate)
        self.dense = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = F.relu(self.batch_norm1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=(2, 1))
        x = F.relu(self.batch_norm2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=(2, 1))
        x = F.relu(self.batch_norm3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=(2, 1))
        
        x = F.relu(self.batch_norm4(self.conv4(x)))
        x = F.max_pool2d(x, kernel_size=(2, 1))
        
        
        x = x.permute(0, 3, 2, 1)
        x = x.reshape(x.shape[0], x.shape[1], x.shape[2] * x.shape[3])

        for rnn_layer in self.rnn_layers:
            x, _ = rnn_layer(x)

        x = self.dropout(x)
        x = self.dense(x)
        x = F.softmax(x, dim = -1)

        return x



model = MyModel(num_classes, rnn_sizes, dropout_rate)
if torch.cuda.is_available():
    model = model.cuda()


In [12]:
summary(model, input_size=(1, 128, 1723), batch_size = -1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 32, 128, 1723]             320
       BatchNorm2d-2        [-1, 32, 128, 1723]              64
            Conv2d-3         [-1, 64, 64, 1723]          18,496
       BatchNorm2d-4         [-1, 64, 64, 1723]             128
            Conv2d-5        [-1, 128, 32, 1723]          73,856
       BatchNorm2d-6        [-1, 128, 32, 1723]             256
            Conv2d-7        [-1, 256, 16, 1723]         295,168
       BatchNorm2d-8        [-1, 256, 16, 1723]             512
               GRU-9  [[-1, 1723, 32], [-1, 2, 16]]               0
          Dropout-10             [-1, 1723, 32]               0
           Linear-11              [-1, 1723, 6]             198
Total params: 388,998
Trainable params: 388,998
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.84
Forw

In [13]:
a.shape, b.shape

(torch.Size([1200, 128, 1723]), torch.Size([1200, 1723, 6]))

In [14]:
model(torch.unsqueeze(a[:10], dim = 1)).shape

torch.Size([10, 1723, 6])

In [15]:
# torch.unsqueeze(a[:10], dim = 0)

In [16]:
arr = np.arange(1200)
np.random.shuffle(arr)
A, B = a[arr], b[arr]
del(a)
del(b)

In [17]:

class MyDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets
        if torch.cuda.is_available():
            self.inputs = self.inputs.cuda()
            self.targets.cuda()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [18]:
# torch.sum(np.transpose(B[0])[0])

In [19]:
weights, total = [], 0
B = B.to("cpu")
for num in range(6):
    tot = 0
    for i in B:
        tot += torch.sum(np.transpose(i)[num])
    total += tot
    weights.append(tot)
weights, total = np.array(weights), np.array([total])[0]
weights = weights/total
print(weights, total)

[0.47619316 0.17411637 0.08655011 0.10461598 0.11719965 0.04132472] 2067600.0


In [20]:
A = A/80 + 1

In [22]:
# def load_checkpoint(filepath):
#     checkpoint = torch.load(filepath)
#     model = MyModel(num_classes, rnn_sizes, dropout_rate)
#     model.load_state_dict(checkpoint)
    
#     return model

# new_model = load_checkpoint("/kaggle/input/saved-model/checkpoint0 (1).pth")
# new_model = new_model.to("cuda")

In [23]:
# new_model

In [24]:
# Define the training loop
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss, bno = 0.0, 0
    metric = MulticlassF1Score(num_classes=6)
    for inputs, targets in train_loader:
        bno += 1
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
#         print(inputs.shape)
        # Forward pass
        outputs = model(torch.unsqueeze(inputs, dim = 1))
        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Update statistics
#         print(outputs.shape)
#         print(targets.shape)
        running_loss += loss.item() * inputs.size(0)
        
#         x, y = torch.reshape(outputs, (outputs.shape[0]*outputs.shape[1], outputs.shape[2])), torch.reshape(torch.argmax(targets, axis = -1), (targets.shape[0]*targets.shape[1],))

        metric.update(torch.reshape(outputs, (outputs.shape[0]*outputs.shape[1], outputs.shape[2])).to("cpu"), torch.reshape(torch.argmax(targets, axis = -1), (targets.shape[0]*targets.shape[1],)).to("cpu") )
        # _, predicted = torch.max(outputs.data, 1)
        # total_predictions += targets.size(0)
        # print(outputs.shape, targets.shape)
        # true_predictions += (predicted == targets).sum().item()
        

    epoch_loss = running_loss / len(train_loader.dataset)
    # epoch_precision = true_predictions / total_predictions
    # epoch_recall = true_predictions / len(train_loader.dataset)

    # return epoch_loss, epoch_precision, epoch_recall
    return epoch_loss, metric.compute()


# Example usage
# Set the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the hyperparameters
epochs = 50
batch_size = 16
learning_rate = 0.002

# Create the model instance
model = MyModel(num_classes, rnn_sizes, dropout_rate).to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create the DataLoader for training dataset
train_dataset = MyDataset(A[:int(0.9*1200)], B[:int(0.9*1200)])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(epochs):
    print(f"epoch number : {epoch + 1}")
    epoch_loss, f_score= train(model, train_loader, criterion, optimizer, device)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, f_score = {f_score} ")


epoch number : 1
Epoch 1/50 - Loss: 2064.0437, f_score = 0.7246232628822327 
epoch number : 2
Epoch 2/50 - Loss: 2023.8475, f_score = 0.8574224710464478 
epoch number : 3
Epoch 3/50 - Loss: 2011.5839, f_score = 0.8917322158813477 
epoch number : 4
Epoch 4/50 - Loss: 2005.5009, f_score = 0.9081732034683228 
epoch number : 5
Epoch 5/50 - Loss: 2002.8429, f_score = 0.915157675743103 
epoch number : 6
Epoch 6/50 - Loss: 2001.8662, f_score = 0.9173491597175598 
epoch number : 7
Epoch 7/50 - Loss: 2000.5858, f_score = 0.9206224083900452 
epoch number : 8
Epoch 8/50 - Loss: 1999.1417, f_score = 0.9253332018852234 
epoch number : 9
Epoch 9/50 - Loss: 1999.2870, f_score = 0.924828052520752 
epoch number : 10
Epoch 10/50 - Loss: 1997.6891, f_score = 0.9299574494361877 
epoch number : 11
Epoch 11/50 - Loss: 1997.2773, f_score = 0.9313702583312988 
epoch number : 12
Epoch 12/50 - Loss: 1996.6985, f_score = 0.933252215385437 
epoch number : 13
Epoch 13/50 - Loss: 1995.7841, f_score = 0.936724245548

In [ ]:
import seaborn as sns
num = 100
sns.heatmap(new_model(torch.unsqueeze(A[num:num+1], dim = 1)).to("cpu").detach().numpy()[0])

In [25]:
torch.save(model.state_dict(), '/kaggle/working/project_base_model.pth')